In [ ]:
%reset -f
import pandas as pd
import numpy as np
import os
from wavhandler import *
import logging
import matplotlib.pyplot as plt
from scipy import signal
import seaborn as sns; sns.set()

plt.rcParams['figure.figsize'] = [14, 6]

logger = logging.getLogger()
logger.setLevel(logging.WARN)

In [ ]:
df = pd.read_csv('mosquitos.csv', delimiter=';')


In [ ]:
plt.subplot(1,2,1)
plt.title('pre-evaluation')
plt.ylim(0,110000)
df['class'].value_counts().plot(kind='bar', color=['r','g','b'])
plt.subplot(1,2,2)
plt.title('post-evaluation')
plt.ylim(0,110000)
df[df.evaluation == 1]['class'].value_counts().plot(kind='bar', color=['g','r','b']) ## HARDCODING COLORS

In [ ]:
z = psd_welch_pows(df[df['class']=='Aedes'].names.tolist())

In [ ]:
sub = pd.DataFrame.from_dict(z, orient='index')

In [ ]:
sub.shape

In [ ]:
mapper = df.set_index('fname').evaluation.to_dict()
sub['evaluation'] = sub.index.map(mapper)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = sub.values[:,:-1]
y = sub.evaluation.values
X_std = StandardScaler().fit_transform(X);  # standardization of data

pca = PCA(n_components=2)
Χ_final = pca.fit_transform(X_std)


In [ ]:
plt.figure(figsize=(15,11))
sns.scatterplot(Χ_final[:,0], Χ_final[:,1], hue=y, alpha=0.5)
#plt.xlabel('component 1')
#plt.ylabel('component 2')


In [ ]:
df_pca = pd.DataFrame(pca.components_,columns=sub.iloc[:,:-1].columns,index = ['PC-1','PC-2'])

In [ ]:
df_pca.loc['PC-1'].plot()

In [ ]:
df_pca.loc['PC-2'].plot()